In [1]:
import pandas as pd

# Load the provided dataset
file_path = r'C:\Users\jianbai\Desktop\protfolio\data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()


,Company,Date,Close/Last,Volume,Open,High,Low
0,AAPL,07/17/2023,$193.99,50520160,$191.90,$194.32,$191.81
1,AAPL,07/14/2023,$190.69,41616240,$190.23,$191.1799,$189.63
2,AAPL,07/13/2023,$190.54,41342340,$190.50,$191.19,$189.78
3,AAPL,07-12-2023,$189.77,60750250,$189.68,$191.70,$188.47
4,AAPL,07-11-2023,$188.08,46638120,$189.16,$189.30,$186.60


In [2]:
# Data cleaning and preparation

# Convert 'Close/Last' to numerical values
data['Close/Last'] = data['Close/Last'].replace('[\$,]', '', regex=True).astype(float)

# Convert 'Date' to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y', errors='coerce')

# Drop rows with NaN values (if any), especially due to date conversion
data = data.dropna()

# Calculate daily returns for each stock
# First, pivot the data to have dates as rows and companies as columns
pivot_data = data.pivot(index='Date', columns='Company', values='Close/Last')

# Calculate daily returns
daily_returns = pivot_data.pct_change().dropna()

# Display the first few rows of daily returns
daily_returns.head()


Company,AAPL,AMD,AMZN,CSCO,META,MSFT,NFLX,QCOM,SBUX,TSLA
Date,,,,,,,,,,
2013-07-19,-0.015765,-0.131466,0.003683,-0.001547,-0.011421,-0.113995,-0.006884,0.005645,0.004527,0.005469
2013-07-22,0.003202,-0.032258,-0.005733,-0.003873,0.006337,0.019427,-0.009890,0.012203,0.003344,0.022974
2013-07-23,-0.017175,-0.061538,-0.007974,-0.006221,0.003264,-0.005936,-0.044663,0.001447,-0.019704,0.002536
2013-07-24,0.051364,-0.008197,-0.007042,0.001174,0.014543,0.004400,-0.035803,-0.014607,-0.015519,-0.008481
2013-07-25,-0.004564,0.019284,0.014919,-0.003517,0.296077,-0.017835,0.022546,0.033067,0.023420,0.019474


In [3]:
# Calculate mean daily returns and covariance matrix
mean_daily_returns = daily_returns.mean()
covariance_matrix = daily_returns.cov()

# Display the mean daily returns and the covariance matrix
mean_daily_returns, covariance_matrix


(Company
 AAPL    0.001908
 AMD     0.003169
 AMZN    0.001754
 CSCO    0.000627
 META    0.002104
 MSFT    0.001706
 NFLX    0.002252
 QCOM    0.000844
 SBUX    0.000918
 TSLA    0.003369
 dtype: float64,
 Company      AAPL       AMD      AMZN      CSCO      META      MSFT      NFLX  \
 Company                                                                         
 AAPL     0.000487  0.000374  0.000283  0.000199  0.000315  0.000281  0.000283   
 AMD      0.000374  0.002140  0.000446  0.000294  0.000469  0.000393  0.000528   
 AMZN     0.000283  0.000446  0.000654  0.000165  0.000394  0.000300  0.000459   
 CSCO     0.000199  0.000294  0.000165  0.000366  0.000171  0.000198  0.000183   
 META     0.000315  0.000469  0.000394  0.000171  0.000948  0.000313  0.000468   
 MSFT     0.000281  0.000393  0.000300  0.000198  0.000313  0.000415  0.000309   
 NFLX     0.000283  0.000528  0.000459  0.000183  0.000468  0.000309  0.001238   
 QCOM     0.000312  0.000540  0.000266  0.000210  0.0002

In [7]:
import numpy as np
from scipy.optimize import minimize

# Set the target expected return as 1.6 times the median of average returns
target_return = 1.6 * mean_daily_returns.median()

# Optimization function using Proximal Gradient Method
def markowitz_optimization(mean_returns, cov_matrix, target_return):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    
    # Constraints and bounds
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights is 1
                   {'type': 'eq', 'fun': lambda x: np.dot(x, mean_returns) - target_return})  # Target return
    bounds = tuple((0,1) for asset in range(num_assets))
    # Additional constraints for AAPL and META (at least 20% each)
    bounds = list(bounds)
    bounds[mean_returns.index.get_loc('AAPL')] = (0.20, 1)
    bounds[mean_returns.index.get_loc('META')] = (0.20, 1)
    bounds = tuple(bounds)

    # Initial guess (equal distribution)
    init_guess = num_assets * [1. / num_assets]

    # Optimization function (minimize the portfolio variance)
    def portfolio_variance(weights, mean_returns, cov_matrix):
        return weights.T @ cov_matrix @ weights

    # Run optimization
    opt_results = minimize(portfolio_variance, init_guess, args=args, 
                           method='SLSQP', bounds=bounds, constraints=constraints)

    return opt_results

# Run the optimization
opt_results = markowitz_optimization(mean_daily_returns, covariance_matrix, target_return)

# Check if the optimization was successful and display the results
if opt_results.success:
    optimal_weights = pd.Series(opt_results.x, index=mean_daily_returns.index)
    optimal_weights
else:
    opt_results.message
    print("optimization failure")


optimization failure


In [9]:
# Adjust the target expected return to 1.2 times the median of average returns
adjusted_target_return = 1.2 * mean_daily_returns.median()

# Adjusted optimization function using Proximal Gradient Method
def adjusted_markowitz_optimization(mean_returns, cov_matrix, target_return):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    
    # Adjusted constraints and bounds
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights is 1
                   {'type': 'ineq', 'fun': lambda x: np.dot(x, mean_returns) - target_return})  # Target return (inequality)
    bounds = tuple((0, 1) for asset in range(num_assets))
    # Adjusted constraints for AAPL and META (at least 10% each)
    bounds = list(bounds)
    bounds[mean_returns.index.get_loc('AAPL')] = (0.10, 1)
    bounds[mean_returns.index.get_loc('META')] = (0.10, 1)
    bounds = tuple(bounds)

    # Initial guess (equal distribution)
    init_guess = num_assets * [1. / num_assets]

    # Optimization function (minimize the portfolio variance)
    def portfolio_variance(weights, mean_returns, cov_matrix):
        return weights.T @ cov_matrix @ weights

    # Run optimization
    opt_results = minimize(portfolio_variance, init_guess, args=args, 
                           method='SLSQP', bounds=bounds, constraints=constraints)

    return opt_results

# Run the adjusted optimization
adjusted_opt_results = adjusted_markowitz_optimization(mean_daily_returns, covariance_matrix, adjusted_target_return)

# Check if the optimization was successful and display the results
if adjusted_opt_results.success:
    adjusted_optimal_weights = pd.Series(adjusted_opt_results.x, index=mean_daily_returns.index)
    adjusted_optimal_weights
else:
    adjusted_opt_results.message


In [11]:
# Re-evaluating the optimization setup

# Ensuring the constraints are set correctly
def reevaluated_markowitz_optimization(mean_returns, cov_matrix, target_return):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    
    # Constraints and bounds (ensuring they are correctly set)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights is 1
                   {'type': 'ineq', 'fun': lambda x: np.dot(x, mean_returns) - target_return})  # Target return (inequality)
    bounds = tuple((0, 1) for asset in range(num_assets))
    # Constraints for AAPL and META (at least 20% each)
    bounds = list(bounds)
    bounds[mean_returns.index.get_loc('AAPL')] = (0.20, 1)
    bounds[mean_returns.index.get_loc('META')] = (0.20, 1)
    bounds = tuple(bounds)

    # Initial guess (adjusted to be more feasible given the constraints)
    init_guess = np.array([0.20 if asset in ['AAPL', 'META'] else (0.6 / (num_assets - 2)) for asset in mean_returns.index])

    # Optimization function (minimize the portfolio variance)
    def portfolio_variance(weights, mean_returns, cov_matrix):
        return weights.T @ cov_matrix @ weights

    # Run optimization using a different method if needed
    opt_results = minimize(portfolio_variance, init_guess, args=args, 
                           method='SLSQP', bounds=bounds, constraints=constraints)

    return opt_results

# Run the re-evaluated optimization
reevaluated_opt_results = reevaluated_markowitz_optimization(mean_daily_returns, covariance_matrix, adjusted_target_return)

# Check if the optimization was successful and display the results
if reevaluated_opt_results.success:
    reevaluated_optimal_weights = pd.Series(reevaluated_opt_results.x, index=mean_daily_returns.index)
    reevaluated_optimal_weights
else:
    reevaluated_opt_results.message


In [13]:
def proximal_gradient_descent(mean_returns, cov_matrix, target_return, max_iter=1000, learning_rate=0.01, threshold=1e-8):
    """
    Proximal Gradient Descent for Markowitz Portfolio Optimization
    """
    num_assets = len(mean_returns)
    weights = np.array([0.20 if asset in ['AAPL', 'META'] else (0.6 / (num_assets - 2)) for asset in mean_returns.index])
    
    # Objective function: Portfolio Variance
    def objective_function(w, cov_matrix):
        return w.T @ cov_matrix @ w

    # Gradient of the objective function
    def gradient(w, cov_matrix):
        return cov_matrix @ w

    # Proximal operation
    def proximal_operation(w, learning_rate):
        w -= learning_rate * gradient(w, cov_matrix)
        w[w < 0] = 0  # Enforce non-negative weights
        w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
        w[mean_returns.index.get_loc('META')] = max(w[mean_returns.index.get_loc('META')], 0.20)  # Enforce META constraint
        w /= np.sum(w)  # Normalize to sum to 1
        return w

    # Proximal gradient descent loop
    for _ in range(max_iter):
        prev_weights = np.copy(weights)
        weights = proximal_operation(weights, learning_rate)

        # Check for convergence
        if np.linalg.norm(weights - prev_weights) < threshold:
            break

    # Ensure the portfolio meets the target return (adjust if necessary)
    if np.dot(weights, mean_returns) < target_return:
        return "Failed to meet the target return"

    return weights

# Run the proximal gradient descent optimization
proximal_weights = proximal_gradient_descent(mean_daily_returns, covariance_matrix, adjusted_target_return)

# Display the results
proximal_weights if isinstance(proximal_weights, str) else pd.Series(proximal_weights, index=mean_daily_returns.index)


C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

'Failed to meet the target return'

In [14]:
# Adjust the target expected return to 0.8 times the median of average returns
adjusted_target_return_2 = 0.8 * mean_daily_returns.median()

# Run the proximal gradient descent optimization with updated target return and constraints
proximal_weights_2 = proximal_gradient_descent(mean_daily_returns, covariance_matrix, adjusted_target_return_2)

# Display the results
proximal_weights_2 if isinstance(proximal_weights_2, str) else pd.Series(proximal_weights_2, index=mean_daily_returns.index)


C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:20: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  w[mean_returns.index.get_loc('AAPL')] = max(w[mean_returns.index.get_loc('AAPL')], 0.20)  # Enforce AAPL constraint
C:\Users\jianbai\AppData\Local\Temp\ipykernel_26260\4147378510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

Company
AAPL    0.204289
AMD     0.072130
AMZN    0.074242
CSCO    0.075789
META    0.202988
MSFT    0.074835
NFLX    0.073457
QCOM    0.074406
SBUX    0.075487
TSLA    0.072378
dtype: float64

In [15]:
# Calculate the expected return, risk (standard deviation), Sharpe ratio, and diversification

# Assumed risk-free rate for Sharpe ratio calculation
risk_free_rate = 0.005

# Calculate the expected portfolio return
expected_portfolio_return = np.dot(proximal_weights_2, mean_daily_returns)

# Calculate the portfolio risk (standard deviation)
portfolio_risk = np.sqrt(np.dot(proximal_weights_2.T, np.dot(covariance_matrix, proximal_weights_2)))

# Calculate the Sharpe ratio
sharpe_ratio = (expected_portfolio_return - risk_free_rate) / portfolio_risk

# Measure of diversification: a simple approach is to look at the distribution of weights
# A more diversified portfolio will have weights more evenly distributed
diversification_measure = proximal_weights_2.std()  # Standard deviation of the weights as a measure of diversification

expected_portfolio_return, portfolio_risk, sharpe_ratio, diversification_measure


(0.0018920738810546454,
 0.01954840245834024,
 -0.15898619468105804,
 0.054636100355096785)

In [18]:
# Creating a table to display the calculated metrics and their values

metrics_table = pd.DataFrame({
    "Metric": ["Expected Return", "Risk Control", "Sharpe Ratio"],
    "Value": [expected_portfolio_return, portfolio_risk, sharpe_ratio]
})

metrics_table


,Metric,Value
0,Expected Return,0.001892
1,Risk Control,0.019548
2,Sharpe Ratio,-0.158986
